# This notebook will be used for the capstone project
## Segmenting and Clustering Neighborhoods in Toronto

### Install requed library

In [1]:
!conda install -c conda-forge Beautifulsoup4 --yes 
!conda install -c conda-forge lxml --yes 
!conda install -c conda-forge html5lib --yes 
!conda install -c conda-forge requests --yes 

Solving environment: ...working... 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-he774522_2
  - anaconda::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-he774522_2
  - anaconda::openssl-1.1.1d-he774522_2, defaults::ca-certificates-2019.8.28-0
  - defaults::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-he774522_2
  - anaconda::openssl-1.1.1d-he774522_2, defaults::ca-certificates-2019.8.28-0
  - anaconda::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-he774522_2
  - defaults::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-he774522_2
  - anaconda::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-he774522_2
  

## Prepare Dataframe

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # library for html scrap

Define procedure for HTML table parse

In [3]:
def parse_html_table(table):
    n_columns = 0
    n_rows=0
    column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):
        
        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)
                
        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                        index= range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1
            
    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass
    
    return df

Dowload Wikipedia page

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)

Take first HTML table from the page and parse it into dataframe

In [5]:
soup = BeautifulSoup(response.text, 'lxml')
df = parse_html_table(soup.find('table'))  

Take a look at the dataframe

In [6]:
df.head()

,Postcode,Borough,Neighborhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


Rename some columns

In [7]:
df.rename(columns={"Neighborhood\n": "Neighborhood"}, inplace=True)

Replace "\n" in the end of the "Neighborhood" column

In [8]:
df['Neighborhood'].replace('\n', '', regex=True, inplace=True)

Drop rows with 'Borough' == 'Not assigned'

In [9]:
df.drop(df[df['Borough'] == 'Not assigned'].index, axis=0, inplace=True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [10]:
df['Neighborhood'].replace('Not assigned', df['Borough'], inplace=True)

Concatenate 'Neighborhood' for one postal code area 

In [11]:
# Group by 'Postcode','Borough'
# Apply concatenation 'Neighborhood' with ','
# Convert to pandas DataFrame
# Reset index
Toronto_df = df.groupby(['Postcode','Borough'], as_index=True)['Neighborhood'].apply(lambda tags: ','.join(tags)).to_frame().reset_index()

Take a look at the result dataframe

In [12]:
Toronto_df.head()

In [0]:
Toronto_df.shape